<h1>Inverse problem</h1>


I expose here how to solve the inverse probleme of reconstructiong $\alpha,\ \beta,\ a$ and $b$ from of finite number of the $u$ solution's values.
The first step is to define a grid to work with and to extract a sample of it, which will be the base for the observation set (see grid.ipynb for more details).

In [10]:
from direct_problem import Grid

grid = Grid(
    x_min=0,
    x_max=5,
    delta_x=0.2,
    t_max=1,
    delta_t=0.01
)

sample_size=200
sample = grid.random_sample(size=sample_size)


The next step is to generate a set of random parameters. Each subset $\alpha_i,\ \beta_i,\ a_i$ and $b_i$ will be used to define and solve a direct problem. 

Each solution, along with those parameters will be used to train a neural network whose objectif is to map solution's values to the parameters.

In [11]:
from data import RandomParameters

parameters_size = 150 # This will also be the numbers of direct 
                      # problems to solve in order to generate the data.

parameters = RandomParameters(parameters_size=parameters_size)

print(parameters)

      alpha      beta         a         b
0  0.369886  0.949827  2.687907  1.964927
1  0.701621  0.127334  0.907685  2.821581
2  0.115038  0.832294  1.988475  0.775089
3  0.327756  0.765553  0.491848  2.910181
4  0.127835  0.555397  2.780775  1.961650


In [12]:
import numpy as np

from data import DataGenerator


f = lambda x: np.maximum(2-x, 0)

data_generator = DataGenerator(grid, parameters, f)

# We build the DataLoader that will be used by pytorch

batch_size = sample_size // 10
train_dl, test_dl = data_generator.dataloader(batch_size=batch_size)

We are now ready to train the model.

In [13]:
from inverse_problem import get_optimized_model, fit

model, opt = get_optimized_model(sample_size, learning_rate=0.005)

epochs = 50

loss_array = fit(model, train_dl, test_dl, epochs, opt) # Also return the average validation loss at 
                                                        # each step of the training
# TODO: Re start from here and write somekind of a dashboard to visualize the whole process

Average validation loss before fit: 1.9164425134658813
Average validation loss: 0.28024375438690186
